In [60]:
from functions import *
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
import pickle
import plotly.express as px
import plotly.graph_objects as go

import dash
from dash import html,Input,Output,dcc
from dash.dependencies import Input, Output, State
import geopandas as gpd
from datetime import date
country_dict = {
"BG":"Bulgaria",
"GR":"Greece",
"HR":"Croatia",
"RO":"Romania",
"RS":"Republic of Serbia",
"SI":"Slovenia",
}
path="../"
def load_augumented(code):
    file = path + 'data/augumented/' + code + '.pkl'
    with open(file, 'rb') as f:
        df = pickle.load(f)
    return df.drop(columns=["Load-1","Load-24"])
def day_split(data_country, features, day):
    data_country=data_country.dropna()
    y = data_country['DayAheadPrice']
    x = data_country[features]
    x_train, x_test, y_train, y_test = x[x.index<day], x.loc[day], y[y.index<day], y.loc[day]
    print("AJLKAJLKAJ")
    return x_train, x_test, y_train, y_test
def train_model_country_day(data_country_input, day, n_features,model_type="rf", model_parameters=None):
    """
    Train a ML model. This function does the following:
        1. rescales the input data
        2. train test split
        3. feature selection on the training data
        4. train the model: either NN or RF
    
    Parameters:
        model_type: 'rf' | 'nn' | 'lr', str
            type of the model to train, 'rf' for random forest, 'lr' for linear regression and 'nn' for neural network
        data_country: DataFrame
            cleaned data for a given country (no Nans etc)
        n_features: int
            number of best features to select
        model_parameters: dict
            dictionary with model parameters, which are parameters for either 
            MLPRegressor or RandomForestRegressor from sklearn
    
    
    Out:
        model: object
            sklearn model
        params: DataFrame
            parameters used for rescaling the data; predictions have to be scaled back 
            for error calculation and out-of-sample new data must be scaled using these params
        selected_features: list or Index, not sure
            features used in training this model
        err: Series
            errors for predictions
    """
    data_country = data_country_input.copy()
    data_country, params = rescale(data_country)
    print("h1")
    x_train, x_test, y_train, y_test = day_split(data_country, data_country.columns[1:],day)
    print("h2")
    selected_features = get_best_features(x_train, n_features)

    x_train = x_train[selected_features]
    x_test = x_test[selected_features]

    if model_type.lower() == 'nn':
        if model_parameters == None:
            # these parameters worked okay for project 1
            model_parameters = {
                'hidden_layer_sizes': (16,16,14)
            }
        model = MLPRegressor(**model_parameters)
    elif model_type.lower() == 'lr':
       model = linear_model.LinearRegression() 
    elif model_type.lower() == 'rf':
        if model_parameters == None:
            # these parameters worked okay for project 1
            model_parameters = {
                'bootstrap': True,
                'min_samples_leaf': 2,
                'n_estimators': 10, 
                'min_samples_split': 3,
                'max_features': 10,
                'max_depth': 10,
                'max_leaf_nodes': None}
            
        model = RandomForestRegressor(**model_parameters)

    model.fit(x_train, y_train)
    predictions = model.predict(x_test)

    predictions = pd.Series(predictions, index=y_test.index).sort_index()
    y_test = y_test.sort_index()


    # to calculate errors, we have to rescale the data back using original parameters
    predictions = predictions * params.loc['DayAheadPrice', 'Std'] + params.loc['DayAheadPrice', 'Mean']
    y_test = y_test * params.loc['DayAheadPrice', 'Std'] + params.loc['DayAheadPrice', 'Mean']
    err = get_errors(y_test, predictions)

    #display(err)
    #plot_predictions(y_test, predictions)
    
    return model, params, selected_features, err, predictions

In [80]:
#train_model_country_day(data_country_input, day, n_features, model_type="rf", model_parameters=None)
from plotly.subplots import make_subplots
def plotDayPrediction(day,n_features,model_type='rf',exclude_features=["Price-1","Price-2"]):
        countries = ["BG", "HR", "RO","SI"]
        countries_clean=[]
        for i,code in enumerate(countries):
                print(i,code)
                try: 
                        test=load_augumented(code).loc[day]
                        print(test.isnull().values.any())
                except: print("Date for this day does not exist") 
                else: 
                        if test.shape[0]==24: countries_clean.append(code)
        print(countries_clean)
        n = len(countries_clean)
        print(n)
        clist = []
        [clist.append(country_dict[x]) for x in countries_clean]
        fig = make_subplots(rows=n, cols=2,
                subplot_titles=clist)

        def predOneDayOneCode(code,row,col):
                df = load_augumented(code).drop(columns=exclude_features)
                print(df.isnull().values.any())
                model, params, selected_features, err, predictions = train_model_country_day(df, day, 12, model_type="rf")
                def rolling(dataframe):
                        return dataframe.rolling(window=roll_window).mean()
                print(selected_features)
                
                
                df = df.loc[day]

                fig.add_trace(go.Scatter(x=df.index, 
                                        y=df["DayAheadPrice"], 
                                        name="Real price",
                                        mode="lines",
                                        line_color="blue",),
                                        row=row,col=col)  
                
                fig.add_trace(go.Scatter(x=predictions.index, 
                                        y=predictions, 
                                        name="Prediction",
                                        mode="lines",
                                        line_color="orange",),
                                        row=row,col=col)  #

                fig.update_layout(
                        #title={'text':'Prediction of Energy Price in %s' % (country_dict[code]),'x':0.5, 'xanchor':'center'},
                        showlegend=False)
                
                #return fig, selected_features, err
                return 0
        #figa=[None]*n
        #sfa=[None]*n
        #erra=[None]*n
        j=1
        k=1
        for i,code in enumerate(countries_clean):
                predOneDayOneCode(code,k,j)
                if k==j: k=k+1
                elif k>j: j=j+1
        #code = "SI"
        #figa, sfa, erra=predOneDayOneCode(code,1,1)
        #print("aaskdjflöaksjdlfkjaskldjfkajdfölaj")
        fig.update_layout(legend=dict(
                orientation="h",
                entrywidth=120,
                yanchor="bottom",
                y=1.02,
                xanchor="right",
                x=1
                ))
        return fig,0,0
figa, sfa, erra = plotDayPrediction("2022-10-04",12)

figa.show()

0 BG
False
1 HR
False
2 RO
False
3 SI
False
['HR', 'RO', 'SI']
3
False
h1
AJLKAJLKAJ
h2
Index(['Month', 'GenFossilGas', 'GenFossilTotal', 'GenOtherRenewable',
       'Temperature', 'GenGeothermal', 'GenWaste', 'Price-24', 'GenHydroRiver',
       'GenHydroReservoir', 'GenTotal', 'GenFossilHardCoal'],
      dtype='object')
False
h1
AJLKAJLKAJ
h2
Index(['PowerUnavailable', 'Month', 'GenNuclear', 'GenFossilGas',
       'GenFossilHardCoal', 'Temperature', 'LoadActual', 'Price-24',
       'GenHydroRiver', 'GenFossilTotal', 'GenFossilBrownCoal', 'GenTotal'],
      dtype='object')
False
h1
AJLKAJLKAJ
h2
Index(['Month', 'GenNuclear', 'GenFossilBrownCoal', 'GenFossilTotal',
       'GenFossilGas', 'Price-24', 'PowerUnavailable', 'GenHydroRiver',
       'Temperature', 'GenTotal', 'ImportTotal', 'ImportAT'],
      dtype='object')


In [77]:
figa.update_layout(height=600, width=300)
figa.show()

In [39]:
rs = load_augumented("BG")
features=['Month', 'Temperature', 'Price-24', 'LoadActual','GenTotal', 'ImportTotal']
day_split(rs, features, "2022-09-03")

AJLKAJLKAJ


(                           Month  Temperature  Price-24  LoadActual  GenTotal  \
 2022-01-02 00:00:00+00:00      1          3.9     80.84      3499.0    5314.0   
 2022-01-02 01:00:00+00:00      1          4.3     84.27      3439.0    5234.0   
 2022-01-02 02:00:00+00:00      1          4.5     88.92      3426.0    5224.0   
 2022-01-02 03:00:00+00:00      1          4.3     73.68      3438.0    5224.0   
 2022-01-02 04:00:00+00:00      1          4.4     77.65      3543.0    5203.0   
 ...                          ...          ...       ...         ...       ...   
 2022-08-26 03:00:00+00:00      8         16.9    387.55      3479.0    5241.0   
 2022-08-26 04:00:00+00:00      8         16.6    168.73      3753.0    5391.0   
 2022-08-26 05:00:00+00:00      8         17.5    314.00      4101.0    5628.0   
 2022-08-26 06:00:00+00:00      8         19.9    340.52      4266.0    5892.0   
 2022-08-26 07:00:00+00:00      8         23.1    214.23      4267.0    5883.0   
 
              

In [70]:
clean=['HR', 'RO', 'SI']
list = []
[list.append(country_dict[x]) for x in clean]
print(list)

['Croatia', 'Romania', 'Slovenia']
